In [1]:
# import os.path as op
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader
import logging
from dataset import DataloaderImg, GreyToRGB, NormalizeVolume, ResizeSample
from torch import optim
import torchvision.transforms.v2 as tfms
from tqdm import tqdm
from torch.utils import tensorboard
from summary import summary_grid
from loss import DiceLoss
from validation import get_validation_loop_len, generalized_dsc, accuracy
import numpy as np
from torchinfo import summary

plt.ion()  # interactive mode

# install pytorch correctly
# https://discuss.pytorch.org/t/torch-cuda-is-not-available/74845/11

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
logging.basicConfig(
    level=logging.INFO,
)
# install cuda driver ubuntu
# https://ubuntu.com/server/docs/nvidia-drivers-installation

cuda


In [2]:
# https://pytorch.org/docs/stable/cuda.html
# torch.cuda.is_initialized()

if torch.cuda.is_available():
    current_device = torch.cuda.current_device()
    print(f"Current cuda device: {torch.cuda.get_device_name(current_device)}")

Current cuda device: NVIDIA RTX A2000


In [3]:
# script for segmentation
# https://github.com/mateuszbuda/brain-segmentation-pytorch

In [4]:
# help in loading images
# https://discuss.pytorch.org/t/how-to-load-all-the-nii-from-the-directory-without-augmentation-using-pytorch-dataloader/60938/3
root_dir = "/media/MeMoSLAP_Subjects/derivatives/automated_electrode_extraction"  #!NOTE: delete "/train" for all subjects

preprocessing = [ResizeSample(), NormalizeVolume(), GreyToRGB()]
transforms = [tfms.RandomRotation(180)]
n_validation = 13
# full_dataset = Dataloder_img('C:/Users/Ali ktk/.spyder-py3/dataloader/data/train/1', 'C:/Users/Ali ktk/.spyder-py3/dataloader/data/train/1/ADNI_136_S_0300_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20080529142830882_S50401_I107759.nii' ,tfms.Compose([tfms.RandomRotation(180).tfms.ToTensor()]))
train_dataset = DataloaderImg(
    root_dir,
    subset="train",
    validation_cases=n_validation,
    preprocessing=preprocessing,
    transforms=transforms,
)
validation_dataset = DataloaderImg(
    root_dir,
    subset="validation",
    validation_cases=n_validation,
    preprocessing=preprocessing,
    transforms=transforms,
)

In [5]:
len(train_dataset), len(validation_dataset)

(48, 13)

In [6]:
unet = torch.hub.load(
    "mateuszbuda/brain-segmentation-pytorch",
    "unet",
    in_channels=3,
    out_channels=1,
    init_features=32,
    pretrained=False,
)

Using cache found in /home/riemanns/.cache/torch/hub/mateuszbuda_brain-segmentation-pytorch_master


Check free GPU memory with `nvtop` in the terminal


In [7]:
unet.to(device)

batch_size = 16
if batch_size > 16:
    raise ValueError("Batch size too large for 6GB of GPU memory.")
epochs = 500
img_freq = 10
lr = 1e-3

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=True)

loaders = {"train": train_loader, "valid": valid_loader}

dsc_loss = DiceLoss()
best_validation_loss = 1

optimizer = optim.Adam(unet.parameters(), lr=lr)

loss_train = []

step = 0
patience = 0

writer = tensorboard.SummaryWriter()

for _ in tqdm(range(epochs), total=epochs):
    for phase in ["train", "valid"]:
        if phase == "valid":
            unet.eval()
        else:
            unet.train()

        for i, data in enumerate(loaders[phase]):
            if phase == "train":
                step += 1

            x, y_true = data
            x, y_true = x.to(device, dtype=torch.float), y_true.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == "train"):
                y_pred = unet(x)
                loss = dsc_loss(y_pred, y_true)

                if phase == "train":
                    loss.backward()
                    optimizer.step()

                writer.add_scalar(f"loss/{phase}", loss.item(), step)

                acc_mean = np.mean(
                    accuracy(
                        y_pred.detach().cpu().numpy(),
                        y_true.detach().cpu().numpy(),
                    )
                )
                writer.add_scalar(f"accuracy/{phase}", acc_mean, step)
            if step % img_freq != 0:
                continue

            writer.add_figure(
                f"step {step}: predictions vs. petra {phase}",
                summary_grid(
                    x.cpu().detach().type(torch.IntTensor), y_pred.cpu().detach()
                ),
                global_step=step,
            )

        if phase == "valid":

            if loss.item() > best_validation_loss:
                patience = 0
                continue

            best_validation_loss = loss.item()
            torch.save(unet.state_dict(), "best_model.pth")

            if best_validation_loss < 0.1:
                patience = +1

            if patience > 10:
                break

writer.flush()

100%|██████████| 500/500 [22:34:01<00:00, 162.48s/it]   


In [ ]:
torch.save(unet, f"unet_epochs_{epochs}_batchsize_{batch_size}_{lr}.pt")

In [7]:
unet = torch.load("unet_epochs_1000_batchsize_15.pt")

In [8]:
mask = np.random.choice([0, 1], size=(4, 1, 288, 288), p=[0.9, 0.1])
pred = mask + np.random.choice([0, 1], size=(4, 1, 288, 288), p=[0.9, 0.1])
pred = pred.clip(0, 1)

accuracy(pred, mask)

array([0.91018036, 0.91027681, 0.90925203, 0.9108314 ])

In [8]:
generalized_dsc(pred, mask)

NameError: name 'pred' is not defined

In [114]:
acc_matrix = np.zeros((len(validation_dataset), validation_dataset.n_slices))
dsc_matrix = np.zeros((len(validation_dataset), validation_dataset.n_slices))

In [8]:
unet.eval()

bs = 4
valloader = DataLoader(validation_dataset, batch_size=bs, shuffle=False)
loop_len = get_validation_loop_len(validation_dataset, bs)

start_idx = 0
idx_step = bs if bs < len(validation_dataset) else len(validation_dataset)

val_slice = validation_dataset.val_slice

for i in tqdm(range(loop_len)):
    for x, y_true in valloader:
        validation_dataset.val_slice = val_slice

        x, y_true = x.to(device, dtype=torch.float), y_true.to(device)
        y_pred = unet(x)

        y_pred, y_true = y_pred.cpu().detach().numpy(), y_true.cpu().detach().numpy()

        acc_matrix[start_idx : start_idx + idx_step, validation_dataset.val_slice] = (
            accuracy(y_pred, y_true)
        )

        dsc_matrix[start_idx : start_idx + idx_step, validation_dataset.val_slice] = (
            generalized_dsc(y_pred, y_true)
        )

        start_idx += bs
        if start_idx >= len(validation_dataset):
            start_idx = 0
            idx_step = bs
            val_slice += 1

  0%|          | 0/224 [00:03<?, ?it/s]


KeyboardInterrupt: 

In [10]:
acc_matrix.mean(axis=1)

NameError: name 'acc_matrix' is not defined

In [138]:
dsc_matrix.mean(axis=1)

array([0.13217167, 0.1242136 , 0.06574694, 0.07564986])